# Saved scripts is assumed to have scripts with the following stuff in it

### breakfast_weekday_1 : Prepare coffee and toast , Eat coffee and toast , Put b1 utensils in sink , Wash b1 utensils
### breakfast_weekday_2 : Prepare cereal and milk , Eat cereal and milk , Put b2 utensils in sink , Wash b2 utensils
### breakfast_weekend : Prepare peanut butter sandwich , Eat sandwich on sofa , Put bw utensils in sink , Wash bw utensils
### icecream : Serve icecream , Eat icecream , Put away icecream , Put ic utensils in sink , Wash ic utensils
### laundry : Collect clothes , Wash clothes , Put away clothes
### meal : Prepare cheese sandwich , Eat cheese sandwich , Return ml utensils , Wash ml utensils
### meal_long : Chop vegetables , Take out chicken , Cook chicken and vegetables , Serve meal , Eat meal , Return lml utensils , Wash lml utensils
### read : Get a book , Read the book , Return the book
### sleep : Get alarm clock and book , Read and sleep

In [ ]:
actions_vacabulary = {
'breakfast_weekday_1' : ['Prepare coffee and toast' , 'Eat coffee and toast' , 'Put b1 utensils in sink' , 'Wash b1 utensils'],
'breakfast_weekday_2' : ['Prepare cereal and milk' , 'Eat cereal and milk' , 'Put b2 utensils in sink' , 'Wash b2 utensils'],
'breakfast_weekend' : ['Prepare peanut butter sandwich' , 'Eat sandwich on sofa' , 'Put bw utensils in sink' , 'Wash bw utensils'],
'icecream' : ['Serve icecream' , 'Eat icecream' , 'Put away icecream' , 'Put ic utensils in sink' , 'Wash ic utensils'],
'laundry' : ['Collect clothes' , 'Wash clothes' , 'Put away clothes'],
'meal' : ['Prepare cheese sandwich' , 'Eat cheese sandwich' , 'Return ml utensils' , 'Wash ml utensils'],
'meal_long' : ['Chop vegetables' , 'Take out chicken' , 'Cook chicken and vegetables' , 'Serve meal' , 'Eat meal' , 'Return lml utensils' , 'Wash lml utensils'],
'read' : ['Get a book' , 'Read the book' , 'Return the book'],
'sleep' : ['Get alarm clock and book' , 'Read and sleep']
}

dt = 10 #min

## Helper class to specify time options for an action

In [ ]:
import numpy as np
import random

class ActionTime:
    def __init__ (self, mean_t, std_t, min_t=None, max_t=None):
        self.mean_t = mean_t
        self.std_t = std_t
        self.min_t = (mean_t-3*std_t) if min_t is None else min_t 
        self.max_t = (mean_t+3*std_t) if max_t is None else max_t 
        assert self.std_t>0, 'Std time should be positive'
        assert self.min_t<self.max_t, 'Minimum time should be less than maximum'

    def sample (self, n=1):
        t = set()
        while len(t) < n:
            val = int(round(np.random.normal(self.mean_t, self.std_t)))
            if val > self.min_t and val < self.max_t:
                t.add(val)
        t=list(t)
        t.sort()
        return t

    def definite (self, n=1):
        return list(np.linspace(self.mean_t-self.std_t, self.mean_t+self.std_t, n))


## Helper functions for time related calculations

In [ ]:
def time_internal(mins, hrs, days=0, weeks=0):
    return int(round(((((weeks*7+days)*24)+hrs)*60+mins)/dt))

def time_external(in_t):
    in_t = in_t*dt
    mins = in_t % 60
    in_t = in_t // 60
    hrs = in_t % 24
    in_t = in_t // 24
    days = in_t % 7
    in_t = in_t // 7
    weeks = in_t
    return(weeks, days, hrs, mins)

## Schedules and constraints

In [ ]:
t=time_internal

weekday_times = []
weekday_times.append([
    {'prob':0.5, 'action':'breakfast_weekday_1', 'time': ActionTime(mean_t=t(0,8), std_t=t(30,0), min_t=t(0,6), max_t=t(0,9))},
    {'prob':0.5, 'action':'breakfast_weekday_2', 'time': ActionTime(mean_t=t(0,8), std_t=t(30,0), min_t=t(0,6), max_t=t(0,9))}
])
weekday_times.append([
    {'prob':0.8, 'action':'meal', 'time': ActionTime(mean_t=t(0,19), std_t=t(0,1), min_t=t(0,18), max_t=t(0,20))},
    {'prob':0.2, 'action':'meal_long', 'time': ActionTime(mean_t=t(0,19), std_t=t(0,1), min_t=t(0,18), max_t=t(0,20))}
])
weekday_times.append([
    {'prob':1.0, 'action':'sleep', 'time': ActionTime(mean_t=t(0,22), std_t=t(30,0), min_t=t(0,21), max_t=t(0,23))}
])


weekend_times = []
weekend_times.append([
    {'prob':0.8, 'action':'breakfast_weekend', 'time': ActionTime(mean_t=t(0,10), std_t=t(30,1), min_t=t(0,8), max_t=t(0,12))},
    {'prob':0.1, 'action':'breakfast_weekday_1', 'time': ActionTime(mean_t=t(0,10), std_t=t(30,1), min_t=t(0,8), max_t=t(0,12))},
    {'prob':0.1, 'action':'breakfast_weekday_2', 'time': ActionTime(mean_t=t(0,10), std_t=t(30,1), min_t=t(0,8), max_t=t(0,12))}
])
weekend_times.append([
    {'prob':0.8, 'action':'read', 'time': ActionTime(mean_t=t(0,12), std_t=t(0,3), min_t=t(0,9), max_t=t(0,16))},
    {'prob':0.2, 'action':'laundry', 'time': ActionTime(mean_t=t(0,12), std_t=t(0,3), min_t=t(0,9), max_t=t(0,16))}
])
weekend_times.append([
    {'prob':0.8, 'action':'meal_long', 'time': ActionTime(mean_t=t(30,13), std_t=t(0,1), min_t=t(0,11), max_t=t(0,15))},
    {'prob':0.2, 'action':'meal', 'time': ActionTime(mean_t=t(30,13), std_t=t(0,1), min_t=t(0,11), max_t=t(0,15))}
])
weekend_times.append([
    {'prob':0.8, 'action':'read', 'time': ActionTime(mean_t=t(0,18), std_t=t(0,3), min_t=t(0,15), max_t=t(0,21))},
    {'prob':0.2, 'action':'laundry', 'time': ActionTime(mean_t=t(0,18), std_t=t(0,3), min_t=t(0,15), max_t=t(0,21))}
])
weekend_times.append([
    {'prob':0.8, 'action':'meal_long', 'time': ActionTime(mean_t=t(30,19), std_t=t(0,1), min_t=t(0,17), max_t=t(0,21))},
    {'prob':0.2, 'action':'meal', 'time': ActionTime(mean_t=t(30,19), std_t=t(0,1), min_t=t(0,17), max_t=t(0,21))}
])
weekend_times.append([
    {'prob':0.7, 'action':'icecream', 'time': ActionTime(mean_t=t(0,20), std_t=t(0,2), min_t=t(0,18), max_t=t(0,22))},
    # 0.3 : None
])
weekend_times.append([
    {'prob':1.0, 'action':'sleep', 'time': ActionTime(mean_t=t(0,23), std_t=t(0,1), min_t=t(0,21), max_t=t(0,24))}
])


constraints = []  #constraint function, error message
constraints.append((lambda sch: all([a[2]<b[2] for a in sch for b in sch if a[0]=='meal' and a[1]==1 and b[0]=='icecream' and b[1]==1]),  'Must eat meal before eating ice cream'))
constraints.append((lambda sch: all([a[2]<b[2] for a in sch for b in sch if a[0]=='meal_long' and a[1]==4 and b[0]=='icecream' and b[1]==1]),  'Must eat long meal before eating ice cream'))
constraints.append((lambda sch: all([a[2]<=b[2] for a in sch for b in sch if b[0]=='sleep' and b[1]==1]),  'Must do everything before sleeping'))
constraints.append((lambda sch: len(set([t[2] for t in sch])) == len([t[2] for t in sch]),  'No two actions can have the same timestamp'))


## Sampling helper functions

In [ ]:
def sample_action_type(probs):
    r = random.random()
    for i,p in enumerate(probs):
        r -= p
        if r<0:
            return i
    return None

def definite_action(_):
    return 0

def get_one_action_with_times(action_options, stochastic=True):
    get_action = sample_action_type if stochastic else definite_action
    action_idx = get_action([a['prob'] for a in action_options])
    if action_idx is None:
        return []
    action = action_options[action_idx]['action']
    action_times = action_options[action_idx]['time'].sample(len(actions_vacabulary[action]))
    return ([(action, i, t) for i,t in enumerate(action_times)])

## Test

In [ ]:
def get_random_schedule(weekend=False):
    template = weekend_times if weekend else weekday_times
    sch = []
    while True:
        for seg in template:
            sch = sch + get_one_action_with_times(seg, stochastic=True)
        good_sch = all([c[0](sch) for c in constraints])
        if good_sch:
            sch.sort(key=lambda x: x[2])
            return sch
        sch = []

sch = get_random_schedule(weekend=True)
for s in (sch):
    print (s[0], s[1],s[2], time_external(s[2]))

for c in constraints:
    if not c[0](sch):
        print(c[1])


## Helper class to realize a schedule

In [ ]:
import json
import os

dataset_dir = 'saved_datasets/my_scripts'

class Routine():
    def __init__(self, sch = None, init_graph_file = '{}/initial_common.json'.format(dataset_dir)):
        self.schedule = sch
        with open (init_graph_file,'r') as f:
            self.init_graph_dict = json.load(init_graph_file)
        self.init_graph = None  # Read dict into internal object
        self.script_paths_and_timestamps()
    
    def script_paths_and_timestamps(self):
        # try:
        #     return self.script_files, self.timestamps
        # except:
        self.script_files = []
        self.timestamps = []
        for seg in self.schedule:
            self.script_files.append(os.path.join(dataset_dir, seg[0], 'parts', seg[1], '.txt'))
            self.timestamps.append(seg[2])
        return self.script_files, self.timestamps

    def execute(self):
        graphs = []
        return self.timestamps, graphs